In [1]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.5 MB/s 
     |████████████████████████████████| 5.5 MB 24.8 MB/s 
     |████████████████████████████████| 1.3 MB 39.6 MB/s 
     |████████████████████████████████| 163 kB 46.3 MB/s 
     |████████████████████████████████| 7.6 MB 16.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=0938d6d76300d3b2b81facb9f19d25567e8368b1984409b517700b3400cd08c0
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
import string

import pandas as pd
from google.colab import drive
import numpy as np
import nltk
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
from nltk.corpus import stopwords
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util, InputExample, losses
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import joblib
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [4]:
# сформируем стоп слова
stop = set(stopwords.words('english'))

# Functions

In [5]:
def clear_string(text: str) -> str:
  """
  Function for cleaning text
  :param text: input text
  """
  text = ''.join([word for word in text if word not in string.punctuation])
  text = text.lower()
  text = ' '.join([word for word in text.split() if word not in stop])
  return text


def find_nearest_idx(query: str, embeddings: np.array, k: int) -> list:
    """ 
    нахождение индексов ближайшых текстов
    :param query: - query
    :param embedding: np.array vectors to find nearest with query
    :param k: number of nearest embeddings to find
    """
    query = clear_string(query)
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, embeddings)[0]
    top_k_ind = torch.topk(cos_scores, K ).indices
    
    return [int(i) for i in top_k_ind]

# Bert Training

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Matching_string_project/Data/cleaned.csv')

In [ ]:
df_train = df.copy()

In [ ]:
df_train.is_duplicate = df_train.is_duplicate.astype('float32')
df_train['name_1'] = df_train['name_1'].astype(str)
df_train['name_2'] = df_train['name_2'].astype(str)

In [ ]:
df_train[:2]

,name_1,name_2,is_duplicate
0,iko industries ltd,enormous industrial trade pvt ltd,0.0
1,apcotex industries ltd,technocraft industries india ltd,0.0


In [ ]:
train, test  = train_test_split(df_train, 
                                stratify=df_train.is_duplicate, 
                                test_size=0.2, 
                                random_state=10)

In [ ]:
# создаем примеры: предложение 1, предложение 2, метка класса
samples = []
for row in range(train.shape[0]):
  label = train.iloc[row][2]
  inp_example = InputExample(texts=[train.iloc[row][0], train.iloc[row][1]], label=label)
  samples.append(inp_example)


In [ ]:
train_dataloader = DataLoader(samples, shuffle=True, batch_size=16)

In [ ]:
torch.cuda.empty_cache()

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

#Define your train dataset, the dataloader and the train loss

train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/24891 [00:00<?, ?it/s]

In [ ]:
joblib.dump(model, '/content/drive/MyDrive/Matching_string_project/Models/sent_bert.joblib')

['/content/drive/MyDrive/Matching_string_project/Models/sent_bert.joblib']

In [ ]:
joblib.load('/content/drive/MyDrive/Matching_string_project/Models/sent_bert.joblib')

In [ ]:
corpus1 = test['name_1'].to_list()
corpus2 = test['name_2'].to_list()

In [ ]:
# создаем эмбеддинги
embedding1 = model.encode(corpus1, convert_to_tensor=True)
embedding2 = model.encode(corpus2, convert_to_tensor=True)

In [ ]:
# создаем список расстояний по каждой паре
cosine_sim=[]

for i in range(len(embedding1)):
    score=util.cos_sim(embedding1[i],embedding2[i])
    cosine_sim.append(score[0].item())

In [ ]:
test['cosine_sim']=cosine_sim

In [ ]:
# преобразовываем расстояния в классы с порогом 0.8
test['pred'] = test['cosine_sim'].apply(lambda x: cosine_to_pred(x, 0.8))

In [ ]:
print('F1_score =', f1_score(test['is_duplicate'], test['pred']))
print('Precision score =', precision_score(test['is_duplicate'], test['pred']))
print('Recall score =', recall_score(test['is_duplicate'], test['pred']))

F1_score = 0.9104046242774566
Precision score = 0.9662576687116564
Recall score = 0.860655737704918


# Make embeddings


In [6]:
df_train = pd.read_csv('/content/drive/MyDrive/Matching_string_project/Data/cleaned.csv')

In [7]:
model = joblib.load('/content/drive/MyDrive/Matching_string_project/Models/sent_bert.joblib')

In [9]:
corpus1 = df_train['name_1'].unique().tolist()

In [ ]:
corpus1

In [11]:
embedding1 = model.encode(corpus1, convert_to_tensor=True)

In [12]:
joblib.dump(embedding1, '/content/drive/MyDrive/Matching_string_project/Data/embedding.csv')

['/content/drive/MyDrive/Matching_string_project/Data/embedding1.csv']

# Work zone

In [52]:
RAND = 10
K = 20

In [43]:
model = joblib.load('/content/drive/MyDrive/Matching_string_project/Models/sent_bert.joblib')
embeddings = joblib.load('/content/drive/MyDrive/Matching_string_project/Data/embedding.csv')
df = pd.read_csv('/content/drive/MyDrive/Matching_string_project/Data/cleaned.csv')

In [41]:
df[df['is_duplicate'] == 0]

,name_1,name_2,is_duplicate
0,iko industries ltd,enormous industrial trade pvt ltd,0
1,apcotex industries ltd,technocraft industries india ltd,0
2,rishichem distributors pvt ltd,dsa,0
3,powermax rubber factory,co one,0
4,tress,longyou industries park zhejiang,0
...,...,...,...
497814,bitmat products,goodyear tire rubber company,0
497815,bnd trading co ltd,zhong yue liang economy trade imp exp co ltd,0
497816,xeikon industrial co ltd dongguan city,yi cheng trading co ltd dongguan city,0
497817,shanghai kechuan trading co ltd,shanghai mg stationery inc,0


In [53]:
%%time
query = 'rishichem distributors pvt ltd' #@param {type:"string"}  
indexes = find_nearest_idx(query, embeddings, K)
print('Возможно мы нашли несколько похожих записей:\n')
for i in indexes:
  print(f'Номер записи: {i}, содержание {df.iloc[i][0]}')

Возможно мы нашли несколько похожих записей:

Номер записи: 2, содержание rishichem distributors pvt ltd
Номер записи: 56019, содержание rishichem distributors pvt ltd
Номер записи: 71562, содержание rishichem distributors pvt ltd
Номер записи: 135294, содержание rishichem distributors pvt ltd
Номер записи: 192622, содержание rishichem distributors pvt ltd
Номер записи: 262173, содержание rishichem distributors pvt ltd
Номер записи: 337666, содержание rishichem distributors pvt ltd
Номер записи: 353457, содержание rishichem distributors pvt ltd
Номер записи: 357928, содержание rishichem distributors pvt ltd
Номер записи: 247433, содержание rishichem distributors pvt ltd
Номер записи: 375818, содержание rishichem distributors pvt ltd
Номер записи: 404860, содержание rishichem distributors pvt ltd
Номер записи: 444709, содержание rishichem distributors pvt ltd
Номер записи: 457394, содержание rishichem distributors pvt ltd
Номер записи: 76691, содержание rishichem mideast ltd
Номер запис